# Learning the shape of manifold from the code generated from celluation from that manifold

## Data generation

### 1. How to generate the stabilizer generators of a code and store them

The stabilizer generators themselves are stored in two matrices $H_z$ and $H_x$, which operate over the field of $\mathbb{F}_2$ 

So, the first step is to code up a class which constructs these matrices for the toric code

In [1]:
# Define a class
class MyClass:
    # Constructor (initialize the object)
    def __init__(self, attribute1, attribute2):
        self.attribute1 = attribute1
        self.attribute2 = attribute2

    # Define methods (functions) for the class
    def method1(self):
        return f"Method 1 called with attributes: {self.attribute1} and {self.attribute2}"

    def method2(self, new_value):
        self.attribute1 = new_value

# Create an instance of the class
obj = MyClass("Value 1", "Value 2")

# Access attributes and call methods
print(obj.attribute1)  # Access an attribute
print(obj.method1())   # Call a method

# Modify attributes
obj.method2("New Value")
print(obj.attribute1)  # Attribute updated

# Create another instance of the class
obj2 = MyClass("Another Value", "Another Value 2")

Value 1
Method 1 called with attributes: Value 1 and Value 2
New Value
